In [1]:
# first check gpu
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-geometric
!pip install torch-geometric-temporal
!pip install torchmetrics

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 110.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=7af08eae1e151c149e6711160094b7a31f05c4c6a685b10142ce90cbf9717771
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.0 MB/s eta 0:00:00
  Prep

In [1]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# graph deep learning
import torch_geometric
import torch_geometric_temporal
import networkx as nx

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch_geometric
%matplotlib inline

# For Progress Bar
from tqdm.auto import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

In [4]:
a = torch.zeros((2, 3))
a is None

False

## get data
+ for x, y in loader shape: (batch_size, in_dim, num_nodes, seq_len)

In [2]:
from Data.generate_data import generate_data, get_data, get_and_normalize_data

train_loader, valid_loader, test_loader, edge_index = generate_data(168, 50, 8)

## train config

In [3]:
from datetime import datetime

train_config = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'n_epochs': 30,  # Number of epochs.
    'learning_rate': 1e-3,
    'early_stop': 400,  # If model has not improved for this many consecutive epochs, stop training.
    'save_path': f'./models/model_{datetime.now()}.ckpt',  # Your model will be saved here.
    'train_criterion_set': ['MAPE', 'MAE', 'RRSE'],
    'valid_criterion_set': ['MAPE', 'MAE', 'RRSE'],
}

## mtgnn without edge

In [4]:
model_args = {'gcn_true': True, 'buildA_true': True, 'gcn_depth': 2, 'num_nodes': 50, 'dropout': 0.3,
              'subgraph_size': 20, 'node_dim': 40, 'dilation_exponential': 2, 'conv_channels': 16,
              'residual_channels': 16, 'skip_channels': 32, 'end_channels': 64, 'seq_in_len': 168, 'in_dim': 7,
              'seq_out_len': 1,
              'layers': 5, 'propalpha': 0.05, 'tanhalpha': 3, 'layer_norm_affline': True, 'kernel_size': 7,
              'kernel_set': [2, 3, 6, 7]}

In [9]:
from model.net import Net

net = Net(model_args, train_config['device'])

In [10]:
loss = nn.MSELoss(reduction='mean').to(train_config['device'])

In [11]:
from model.criterions import MapeCriterion, MaeCriterion, RrseCriterion

train_criterion_set = [MapeCriterion(train_config['device']), MaeCriterion(train_config['device']), RrseCriterion(train_config['device'])]

valid_criterion_set = [MapeCriterion(train_config['device']), MaeCriterion(train_config['device']), RrseCriterion(train_config['device'])]

In [12]:
optim = torch.optim.Adam(net.parameters())

In [ ]:
from model.log import Log

log = Log(train_config)

## train model

In [21]:
from model.modle import Model
model = Model(net, loss, train_criterion_set, valid_criterion_set, optim, log, train_config)

In [22]:
model.trainer(train_loader, valid_loader)